In [1]:
import re
import string
import pickle
import keras
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.tokenize import TweetTokenizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Activation
from keras.layers import Conv2D, MaxPool2D, Reshape
from keras.layers import Input, concatenate, Dropout
from keras.layers import Embedding, Concatenate
from keras.optimizers import Adam, SGD, RMSprop
from keras import optimizers
from keras import regularizers
from keras.utils.vis_utils import plot_model
from nltk.corpus import stopwords


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary

In [2]:
train = pd.read_csv('./train_en.tsv', delimiter='\t',encoding='utf-8')
dev = pd.read_csv('./dev_en.tsv', delimiter='\t',encoding='utf-8')

In [3]:
def clean_tweets(tweet):
    tweet = re.sub('@(\\w{1,15})\b', '', tweet)
    tweet = tweet.replace("via ", "")
    tweet = tweet.replace("RT ", "")
    tweet = tweet.lower()
    return tweet
    
def clean_url(tweet):
    tweet = re.sub('http\\S+', '', tweet, flags=re.MULTILINE)   
    return tweet
    
def remove_stop_words(tweet):
    stops = set(stopwords.words("english"))
    stops.update(['.',',','"',"'",'?',':',';','(',')','[',']','{','}'])
    toks = [tok for tok in tweet if not tok in stops and len(tok) >= 3]
    return toks
    
def stemming_tweets(tweet):
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in tweet]
    return stemmed_words

def remove_number(tweet):
    newTweet = re.sub('\\d+', '', tweet)
    return newTweet

def remove_hashtags(tweet):
    result = ''

    for word in tweet.split():
        if word.startswith('#') or word.startswith('@'):
            result += word[1:]
            result += ' '
        else:
            result += word
            result += ' '

    return result

In [4]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/aniket.joshi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
def preprocessing(tweet, swords = True, url = True, stemming = False, ctweets = True, number = True, hashtag = True):

    if ctweets:
        tweet = clean_tweets(tweet)

    if url:
        tweet = clean_url(tweet)

    if hashtag:
        tweet = remove_hashtags(tweet)
    
    twtk = TweetTokenizer(strip_handles=True, reduce_len=True)

    if number:
        tweet = remove_number(tweet)
    
    tokens = [w.lower() for w in twtk.tokenize(tweet) if w != "" and w is not None]

    if swords:
        tokens = remove_stop_words(tokens)

    if stemming:
        tokens = stemming_tweets(tokens)

    text = " ".join(tokens)
#     print(text)
    return text

In [19]:
train_text  = train['text'].map(lambda x: preprocessing(x, swords = True, url = True, stemming = False, ctweets = True, number = False, hashtag = False))
y_train  = train['HS']
id_train = train['id']
ag_train = train['AG']
test_text  = dev['text'].map(lambda x: preprocessing(x, swords = True, url = True, stemming = False, ctweets = True, number = False, hashtag = False))
y_test  = dev['HS']
id_test = dev['id']
ag_test = dev['AG']

data = np.concatenate((train_text, test_text), axis=0)
classes = np.concatenate((y_train, y_test), axis=0)


In [20]:
classes = []
for index,word in enumerate(train_text):
#     print(y_train[index],ag_train[index])
    if y_train[index]:
        if ag_train[index]:
            classes.append(0)
#             print(0)
        else:
            classes.append(1)
    else:
        classes.append(2)

y_train = classes
classes = []
for index,word in enumerate(test_text):
    if y_test[index]:
        if ag_test[index]:
            classes.append(0)
        else:
            classes.append(1)
    else:
        classes.append(2)
y_test = classes

In [21]:
from keras.utils import np_utils
count1 = 0
count2 = 0
count3 = 0
for x in y_train:
    if x==0:
        count1+=1
    elif x == 1:
        count2+=1
    else:
        count3+=1

In [22]:
count1,count2,count3

(1559, 2224, 5217)

In [23]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [24]:
y_train[:10]

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [25]:
def word_embeddings(word_index, num_words, word_embedding_dim):
    embeddings_index = {}
    
    f = open('./glove.6B.300d.txt', 'r', encoding='utf-8')
    
    for line in tqdm(f):
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    
    f.close()

    matrix = np.zeros((num_words, word_embedding_dim))
    
    for word, i in word_index.items():
        if i >= max_features:
            continue
        
        embedding_vector = embeddings_index.get(word)
        
        if embedding_vector is not None:
            matrix[i] = embedding_vector

    return matrix

In [17]:
embedding_dim = 300
max_features = 25000
maxlen = 40
batch_size = 128
epochs = 5
filter_sizes = [3,4]
num_filters = 512
drop = 0.5

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(data)

X = tokenizer.texts_to_sequences(train_text)
Y = tokenizer.texts_to_sequences(test_text)

tweets = sequence.pad_sequences(X, maxlen=maxlen)
x_test = sequence.pad_sequences(Y, maxlen=maxlen)

word_index = tokenizer.word_index

num_words = min(max_features, len(word_index) + 1)

In [26]:
x_train, x_val, y_train, y_val = train_test_split(tweets, y_train, test_size=0.1, random_state=None)


In [16]:
from tqdm import tqdm
embedding_matrix = word_embeddings(word_index, num_words, embedding_dim)

400000it [00:26, 14824.52it/s]


In [27]:
tweet_input = Input(shape=(maxlen,), dtype='int32')

embedding = Embedding(num_words, embedding_dim, weights=[embedding_matrix], input_length=maxlen, trainable=True)(tweet_input)

reshape = Reshape((maxlen, embedding_dim, 1))(embedding)

cnn1 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='tanh')(reshape)

max1 = MaxPool2D(pool_size=(maxlen - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(cnn1)

cnn2 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal', activation='tanh')(reshape)

max2 = MaxPool2D(pool_size=(maxlen - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(cnn2)


concatenated_tensor = Concatenate(axis=1)([max1, max2])

flatten = Flatten()(concatenated_tensor)

dropout = Dropout(drop)(flatten)

dens = Dense(num_filters, activation='relu')(dropout)

output = Dense(3, activation='softmax')(dens)

model = Model(inputs=tweet_input, outputs=output)

opt = RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.01)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

model.summary()



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 40)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 40, 300)      5310000     input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 40, 300, 1)   0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 38, 1, 512)   461312      reshape_1[0][0]                  
__________________________________________________________________________________________________
conv2d_2 (

In [28]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]
model.fit(x_train, y_train, batch_size=batch_size, epochs=5, shuffle=True, callbacks = callbacks_list,validation_data=(x_val, y_val))



Train on 8100 samples, validate on 900 samples
Epoch 1/5
8100/8100 [==============================] - 10s 1ms/step - loss: 1.8606 - acc: 0.5691 - val_loss: 0.9449 - val_acc: 0.6056
Epoch 2/5
8100/8100 [==============================] - 9s 1ms/step - loss: 0.7962 - acc: 0.6514 - val_loss: 0.8973 - val_acc: 0.5711
Epoch 3/5
8100/8100 [==============================] - 9s 1ms/step - loss: 0.6709 - acc: 0.7151 - val_loss: 0.7729 - val_acc: 0.6789
Epoch 4/5
8100/8100 [==============================] - 9s 1ms/step - loss: 0.5747 - acc: 0.7616 - val_loss: 0.7988 - val_acc: 0.6600
Epoch 5/5
8100/8100 [==============================] - 9s 1ms/step - loss: 0.4737 - acc: 0.8091 - val_loss: 0.7768 - val_acc: 0.6933


In [30]:
a = (model.predict(x_test, batch_size=batch_size))

In [31]:
a[:10]

array([[1.0408656e-01, 2.2348827e-01, 6.7242521e-01],
       [1.0056634e-02, 8.6935405e-03, 9.8124987e-01],
       [2.7781087e-01, 5.4567772e-01, 1.7651132e-01],
       [2.8649214e-01, 4.0063259e-01, 3.1287524e-01],
       [3.7587533e-04, 4.6927592e-04, 9.9915481e-01],
       [2.0718593e-02, 8.6163975e-02, 8.9311743e-01],
       [1.1680706e-01, 2.0619969e-01, 6.7699319e-01],
       [6.8230946e-03, 9.4031125e-02, 8.9914584e-01],
       [3.3007205e-02, 1.3352007e-01, 8.3347273e-01],
       [1.9852951e-01, 5.5519730e-01, 2.4627320e-01]], dtype=float32)

In [32]:
y_pred = (a == a.max(axis=1)[:,None]).astype(int)

In [33]:
y_pred

array([[0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]])

In [34]:
from sklearn.metrics import f1_score, precision_score, accuracy_score, recall_score

print("F1.........: %f" %(f1_score(y_test, y_pred, average="macro")))
print("Precision..: %f" %(precision_score(y_test, y_pred, average="macro")))
print("Recall.....: %f" %(recall_score(y_test, y_pred, average="macro")))
print("Accuracy...: %f" %(accuracy_score(y_test, y_pred)))

F1.........: 0.519361
Precision..: 0.516387
Recall.....: 0.525887
Accuracy...: 0.585000


In [35]:
import sklearn.metrics as metrics
import pylab as pl
import matplotlib.pyplot as plt
labels = [0, 1,2]
cm = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1), labels)
print(cm)
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
plt.title('Confusion matrix of the classifier')
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

[[100  62  42]
 [ 58  88  77]
 [ 75 101 397]]


<Figure size 640x480 with 2 Axes>

In [36]:
print(metrics.classification_report(y_test, y_pred, digits=3))

              precision    recall  f1-score   support

           0      0.429     0.490     0.458       204
           1      0.351     0.395     0.371       223
           2      0.769     0.693     0.729       573

   micro avg      0.585     0.585     0.585      1000
   macro avg      0.516     0.526     0.519      1000
weighted avg      0.607     0.585     0.594      1000
 samples avg      0.585     0.585     0.585      1000



In [43]:
x1 = preprocessing("I am going to kill them with a gun")
x1 = [x1]
y1 = tokenizer.texts_to_sequences(x1)
tweets1 = sequence.pad_sequences(y1, maxlen=maxlen)
model.predict(tweets1)

array([[0.15625197, 0.15428667, 0.6894614 ]], dtype=float32)

In [235]:
for index,word in enumerate(dev['text']):
    print(word,y_pred[index],y_test[index])
    print("\n")
    break

I swear I’m getting to places just in the nick of time! It’s exhausting @Sam_Schulman RT @MaireadEvvoMc: Made your bed, now lie in the flea pit you created 'Tourists go home, refugees welcome': why Barcelona chose migrants over visitors https://t.co/gtLMcuFC5p [0 0 1] [0. 0. 1.]


